In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import tensorflow as tf
import torch

print("NumPy:", np.__version__)
print("Pandas:", pd.__version__)
print("Matplotlib:", plt.matplotlib.__version__)
print("Scikit-learn:", sklearn.__version__)
print("TensorFlow:", tf.__version__)
print("PyTorch:", torch.__version__)


NumPy: 1.26.4
Pandas: 2.3.0
Matplotlib: 3.10.3
Scikit-learn: 1.5.2
TensorFlow: 2.16.2
PyTorch: 2.3.1+cpu
